### For getting personwise keypoints

In [61]:
import cv2
import time
import numpy as np
from random import randint

protoFile = "../resources/pose/coco/pose_deploy_linevec.prototxt"
weightsFile = "../resources/pose/coco/pose_iter_440000.caffemodel"
nPoints = 18
# COCO Output Format
keypointsMapping = ['Nose', 'Neck', 'R-Sho', 'R-Elb', 'R-Wr', 'L-Sho', 
                    'L-Elb', 'L-Wr', 'R-Hip', 'R-Knee', 'R-Ank', 'L-Hip', 
                    'L-Knee', 'L-Ank', 'R-Eye', 'L-Eye', 'R-Ear', 'L-Ear']

POSE_PAIRS = [[1,2], [1,5], [2,3], [3,4], [5,6], [6,7],
              [1,8], [8,9], [9,10], [1,11], [11,12], [12,13],
              [1,0], [0,14], [14,16], [0,15], [15,17],
              [2,17], [5,16] ]

# index of pafs correspoding to the POSE_PAIRS
# e.g for POSE_PAIR(1,2), the PAFs are located at indices (31,32) of output, Similarly, (1,5) -> (39,40) and so on.
mapIdx = [[31,32], [39,40], [33,34], [35,36], [41,42], [43,44], 
          [19,20], [21,22], [23,24], [25,26], [27,28], [29,30], 
          [47,48], [49,50], [53,54], [51,52], [55,56], 
          [37,38], [45,46]]

colors = [ [0,100,255], [0,100,255], [0,255,255], [0,100,255], [0,255,255], [0,100,255],
         [0,255,0], [255,200,100], [255,0,255], [0,255,0], [255,200,100], [255,0,255],
         [0,0,255], [255,0,0], [200,200,0], [255,0,0], [200,200,0], [0,0,0]]

net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)


def getPoseData(image1):
    ###########################################
    # Find the Keypoints using Non Maximum Suppression on the Confidence Map
    def getKeypoints(probMap, threshold=0.1):

        mapSmooth = cv2.GaussianBlur(probMap,(3,3),0,0)

        mapMask = np.uint8(mapSmooth>threshold)
        keypoints = []

        #find the blobs
        _, contours, _ = cv2.findContours(mapMask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        #for each blob find the maxima
        for cnt in contours:
            blobMask = np.zeros(mapMask.shape)
            blobMask = cv2.fillConvexPoly(blobMask, cnt, 1)
            maskedProbMap = mapSmooth * blobMask
            _, maxVal, _, maxLoc = cv2.minMaxLoc(maskedProbMap)
            keypoints.append(maxLoc + (probMap[maxLoc[1], maxLoc[0]],))

        return keypoints

    # Find valid connections between the different joints of a all persons present
    def getValidPairs(output):
        valid_pairs = []
        invalid_pairs = []
        n_interp_samples = 10
        paf_score_th = 0.1
        conf_th = 0.7
        # loop for every POSE_PAIR
        for k in range(len(mapIdx)):
            # A->B constitute a limb
            pafA = output[0, mapIdx[k][0], :, :]
            pafB = output[0, mapIdx[k][1], :, :]
            pafA = cv2.resize(pafA, (frameWidth, frameHeight))
            pafB = cv2.resize(pafB, (frameWidth, frameHeight))

            # Find the keypoints for the first and second limb
            candA = detected_keypoints[POSE_PAIRS[k][0]]
            candB = detected_keypoints[POSE_PAIRS[k][1]]
            nA = len(candA)
            nB = len(candB)

            # If keypoints for the joint-pair is detected
            # check every joint in candA with every joint in candB 
            # Calculate the distance vector between the two joints
            # Find the PAF values at a set of interpolated points between the joints
            # Use the above formula to compute a score to mark the connection valid

            if( nA != 0 and nB != 0):
                valid_pair = np.zeros((0,3))
                for i in range(nA):
                    max_j=-1
                    maxScore = -1
                    found = 0
                    for j in range(nB):
                        # Find d_ij
                        d_ij = np.subtract(candB[j][:2], candA[i][:2])
                        norm = np.linalg.norm(d_ij)
                        if norm:
                            d_ij = d_ij / norm
                        else:
                            continue
                        # Find p(u)
                        interp_coord = list(zip(np.linspace(candA[i][0], candB[j][0], num=n_interp_samples),
                                                np.linspace(candA[i][1], candB[j][1], num=n_interp_samples)))
                        # Find L(p(u))
                        paf_interp = []
                        for k in range(len(interp_coord)):
                            paf_interp.append([pafA[int(round(interp_coord[k][1])), int(round(interp_coord[k][0]))],
                                               pafB[int(round(interp_coord[k][1])), int(round(interp_coord[k][0]))] ]) 
                        # Find E
                        paf_scores = np.dot(paf_interp, d_ij)
                        avg_paf_score = sum(paf_scores)/len(paf_scores)

                        # Check if the connection is valid
                        # If the fraction of interpolated vectors aligned with PAF is higher then threshold -> Valid Pair  
                        if ( len(np.where(paf_scores > paf_score_th)[0]) / n_interp_samples ) > conf_th :
                            if avg_paf_score > maxScore:
                                max_j = j
                                maxScore = avg_paf_score
                                found = 1
                    # Append the connection to the list
                    if found:            
                        valid_pair = np.append(valid_pair, [[candA[i][3], candB[max_j][3], maxScore]], axis=0)

                # Append the detected connections to the global list
                valid_pairs.append(valid_pair)
            else: # If no keypoints are detected
                #print("No Connection : k = {}".format(k))
                invalid_pairs.append(k)
                valid_pairs.append([])
        #print(valid_pairs)
        return valid_pairs, invalid_pairs

    # This function creates a list of keypoints belonging to each person
    # For each detected valid pair, it assigns the joint(s) to a person
    # It finds the person and index at which the joint should be added. This can be done since we have an id for each joint
    def getPersonwiseKeypoints(valid_pairs, invalid_pairs):
        # the last number in each row is the overall score 
        personwiseKeypoints = -1 * np.ones((0, 19))

        for k in range(len(mapIdx)):
            if k not in invalid_pairs:
                partAs = valid_pairs[k][:,0]
                partBs = valid_pairs[k][:,1]
                indexA, indexB = np.array(POSE_PAIRS[k])

                for i in range(len(valid_pairs[k])): 
                    found = 0
                    person_idx = -1
                    for j in range(len(personwiseKeypoints)):
                        if personwiseKeypoints[j][indexA] == partAs[i]:
                            person_idx = j
                            found = 1
                            break

                    if found:
                        personwiseKeypoints[person_idx][indexB] = partBs[i]
                        personwiseKeypoints[person_idx][-1] += keypoints_list[partBs[i].astype(int), 2] + valid_pairs[k][i][2]

                    # if find no partA in the subset, create a new subset
                    elif not found and k < 17:
                        row = -1 * np.ones(19)
                        row[indexA] = partAs[i]
                        row[indexB] = partBs[i]
                        # add the keypoint_scores for the two keypoints and the paf_score 
                        row[-1] = sum(keypoints_list[valid_pairs[k][i,:2].astype(int), 2]) + valid_pairs[k][i][2]
                        personwiseKeypoints = np.vstack([personwiseKeypoints, row])
        return personwiseKeypoints


    ###########################################
    frameWidth = image1.shape[1]
    frameHeight = image1.shape[0]

    t = time.time()
    

    # Fix the input Height and get the width according to the Aspect Ratio
    inHeight = 368
    inWidth = int((inHeight/frameHeight)*frameWidth)

    inpBlob = cv2.dnn.blobFromImage(image1, 1.0 / 255, (inWidth, inHeight),
                              (0, 0, 0), swapRB=False, crop=False)

    net.setInput(inpBlob)
    output = net.forward()
    detected_keypoints = []
    keypoints_list = np.zeros((0,3))
    keypoint_id = 0
    threshold = 0.1
    
    #--- Also save keypoints in a dictionary
    keypoints_dict = {}

    for part in range(nPoints):
        probMap = output[0,part,:,:]
        probMap = cv2.resize(probMap, (image1.shape[1], image1.shape[0]))

        keypoints = getKeypoints(probMap, threshold)
        #print("Keypoints - {} : {}".format(keypointsMapping[part], keypoints))
        keypoints_with_id = []
        for i in range(len(keypoints)):
            keypoints_with_id.append(keypoints[i] + (keypoint_id,))
            keypoints_list = np.vstack([keypoints_list, keypoints[i]])
            keypoint_id += 1

        detected_keypoints.append(keypoints_with_id)
        
    for kpointcollection in detected_keypoints:
        if len(kpointcollection)>0:
            for kpoint in kpointcollection:
                keypoints_dict[kpoint[3]] = [kpoint[0], kpoint[1]]

    #frameClone = image1.copy()
    #for i in range(nPoints):
    #    for j in range(len(detected_keypoints[i])):
    #        cv2.circle(frameClone, detected_keypoints[i][j][0:2], 3, [0,0,255], -1, cv2.LINE_AA)
    #plt.figure(figsize=[15,15])
    #plt.imshow(frameClone[:,:,[2,1,0]])

    valid_pairs, invalid_pairs = getValidPairs(output)
    personwiseKeypoints = getPersonwiseKeypoints(valid_pairs, invalid_pairs)

    #for i in range(17):
    #    for n in range(len(personwiseKeypoints)):
    #        index = personwiseKeypoints[n][np.array(POSE_PAIRS[i])]
    #        if -1 in index:
    #            continue
    #        B = np.int32(keypoints_list[index.astype(int), 0])
    #        A = np.int32(keypoints_list[index.astype(int), 1])
    #        cv2.line(frameClone, (B[0], A[0]), (B[1], A[1]), colors[i], 3, cv2.LINE_AA)
    
    personCoords = {}
    count = 0
    for person in personwiseKeypoints:
        getPersonArray = {}
        if (person[0:19]==-1.).sum() <=6:
            for i in range(18):
                if person[i] == -1:
                    getPersonArray[i] = [-1, -1]
                else:
                    getPersonArray[i] = keypoints_dict[person[i]]
            personCoords["person"+str(count)] = getPersonArray
        count += 1
    
    return personCoords

def getVideoData(videoDir, videoFile, videoID, start_frame=1, end_frame=-1, step=1):
    cap = cv2.VideoCapture(videoDir + videoFile)
    #success,image = cap.read()
    file_to_write = open("../data/frames/"+videoFile[:-4]+"_"+str(start_frame)+".json", "w")
    
    count = 1
    success = True
    while success:
        success,image = cap.read()
        if count >= start_frame and count%step==0 and success:
            persons = getPoseData(image)
            
            if len(persons) >0:
                frame = {
                    "videoFile":videoFile,
                    "videoID":videoID,
                    "frame_no":count,
                    "pers_coords": persons,
                    "person_count": len(persons.keys())
                }
            
                file_to_write.write(json.dumps(frame)+"\n")
            
        count += 1
        
        if end_frame != -1 and count>end_frame:
            break
        
    file_to_write.close()
    
    
    
def getVideoData_List(videoDir, videoFile, videoID, list_frames=[]):
    cap = cv2.VideoCapture(videoDir + videoFile)
    #success,image = cap.read()
    file_to_write = open("../data/frames/"+videoFile[:-4]+"_"+str(videoID)+".json", "w")
    count=0
    for i in list_frames:
        success,image = cap.read()
        count += 1
        if i == 1 and success:
            persons = getPoseData(image)
            
            if len(persons) >0:
                frame = {
                    "videoFile":videoFile,
                    "videoID":videoID,
                    "frame_no":count,
                    "pers_coords": persons,
                    "person_count": len(persons.keys())
                }
            
                file_to_write.write(json.dumps(frame)+"\n")        
    file_to_write.close()

def getFrames(file_name, file_output, start_frame=0, end_frame=20, step=1):
    cap = cv2.VideoCapture(file_name)
    success,image = cap.read()
    count = 0
    success = True
    while success:
        success,image = cap.read()
        if count > start_frame and count%step==0:
            cv2.imwrite(file_output+"%d.jpg" % count, image)     # save frame as JPEG file
            print('Read a new frame: ', success)
        if count>end_frame:
            break
        count += 1

In [23]:
#vidDir = "../data/vids/"
#getFrames(vidDir+"QSgS8RIFZQA.mp4", vidDir+"QSgS8RIFZQA", start_frame=1440, end_frame=1452)

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True


In [32]:
#img_file = "../data/vids/QSgS8RIFZQA1449.jpg"
#image1 = cv2.imread(img_file)
#dictPoints = getPoseData(image1)
#print(dictPoints)
vidDir = "../data/vids/"
getVideoData(vidDir, "QSgS8RIFZQA.mp4", "QSgS8RIFZQA", start_frame=1220, end_frame=1230)
getVideoData(vidDir, "QSgS8RIFZQA.mp4", "QSgS8RIFZQA", start_frame=1584, end_frame=1600)

In [14]:
print(json.dumps(dictPoints))

{"person0": {"0": [565, 66], "1": [573, 89], "2": [565, 88], "3": [-1, -1], "4": [-1, -1], "5": [581, 89], "6": [581, 113], "7": [573, 136], "8": [559, 129], "9": [564, 160], "10": [566, 191], "11": [573, 130], "12": [573, 160], "13": [573, 191], "14": [-1, -1], "15": [566, 66], "16": [-1, -1], "17": [575, 66]}, "person1": {"0": [215, 51], "1": [183, 81], "2": [197, 82], "3": [247, 105], "4": [285, 90], "5": [-1, -1], "6": [-1, -1], "7": [-1, -1], "8": [168, 176], "9": [160, 253], "10": [136, 315], "11": [153, 175], "12": [197, 238], "13": [182, 308], "14": [214, 49], "15": [-1, -1], "16": [199, 50], "17": [-1, -1]}, "person2": {"0": [66, 57], "1": [67, 73], "2": [57, 73], "3": [51, 97], "4": [75, 90], "5": [82, 73], "6": [89, 89], "7": [66, 91], "8": [59, 120], "9": [66, 152], "10": [66, 183], "11": [81, 114], "12": [81, 152], "13": [81, 183], "14": [66, 50], "15": [73, 50], "16": [58, 51], "17": [75, 51]}, "person3": {"0": [627, 57], "1": [627, 66], "2": [612, 66], "3": [605, 89], "4

In [65]:
source="../data/vids/vid_source1.json"
vidDir = "../data/vids/"

with open(source, "r") as f:
    for line in f:
        #print(line)
        dict_source = json.loads(line)
        #print(dict_source)
        name = dict_source['filename']
        ID = dict_source['ID']
        len_times = len(list(dict_source["times"].keys()))
        save_frame = []
        step = 3
        for t in range(1, len_times+1):
            frameTimes = dict_source["times"][str(t)]
            start = (frameTimes[0][0]*60 + frameTimes[0][1])*24
            end = (frameTimes[1][0]*60 + frameTimes[1][1])*24
            
            save_frame += ((start-len(save_frame))*[0])
            x = list(map(lambda x: int(x%step==0) , range(0, end-start)))
            save_frame += x
        #print((save_frame))
        getVideoData_List(vidDir, name, ID, list_frames = save_frame)

KeyboardInterrupt: 

In [46]:
start = 1200
end = 1324
step = 3
n = int((end-start)/step)
rem = (start-end)%step
print(f"start: {start}, end:{end}, n:{n}, rem:{rem}")
x = list(map(lambda x: int(x%step==0) , range(0, 1324-1200)))
print(x)


start: 1200, end:1324, n:41, rem:2
[1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1]
